In [10]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import random

from baseline import exhaustive, random_placement, heft, random_op_greedy_dev, random_op_est_dev
from os import listdir
from os.path import isfile, join
from baseline import random_placement, get_placement_constraints
from env.utils import generate_graph, generate_network
from env.network import StarNetwork, FullNetwork
from env.program import Program
from env.latency import evaluate, computation_latency, communication_latency, simulate

import os, itertools
import pickle, json

import time


In [14]:
number_of_tests = 20

logdir='runs/2022-05-03_12-53-47_rl_op'

data_id = pickle.load(open(join(logdir, 'data_para.pkl'), 'rb'))
test_programs = data_id['test_program']
test_networks = data_id['test_network']

print(test_programs)
print(test_networks)


a = list(itertools.product(test_programs.keys(), test_networks.keys()))
p_id, n_id = [s[0] for s in a], [s[1] for s in a]

norm_serial_min = []
norm_serial_avg = []

for program_id in p_id:
    for network_id in n_id:
        net_data = generate_network(*list(test_networks[network_id].values()))
        net = FullNetwork(net_data['delay'], net_data['comm_speed'], net_data['speed'], net_data['device_constraints'])
        prg = Program(generate_graph(*list(test_programs[program_id].values())))
        
        constraints = get_placement_constraints(prg, net)
        
        comp_min = []
        comm_min = []
        
        comp_a = []
        comm_a = []
        
        for n in prg.P.nodes():
            comp_t = [computation_latency(prg, net, n, dev) for dev in constraints[n]]
            comp_min.append(min(comp_t))
            comp_a.append(np.mean(comp_t))
            
        for e in prg.P.edges():
            d1 = constraints[e[0]]
            d2 = constraints[e[1]]
            comm_t = [communication_latency(prg, net, e[0],e[1], dev1, dev2) for dev1 in d1 for dev2 in d2]
            comm_min.append(min(comm_t))
            comm_a.append(np.mean(comm_t))
        norm_serial_min.append(np.sum(comp_min) + np.sum(comm_min))
        norm_serial_avg.append(np.sum(comp_a) + np.sum(comm_a))
print(norm_serial_min)
print(norm_serial_avg)

{0: {'alpha': 0.1, 'v': 20, 'connect_prob': 0.2, 'seed': 20, 'num_types': 5, 'avg_compute': 100, 'avg_bytes': 100, 'b_comp': 0.2, 'b_comm': 0.2}}
{0: {'n_devices': 20, 'seed': 20, 'num_types': 5, 'type_prob': 0.2, 'avg_speed': 5, 'avg_bw': 100, 'avg_delay': 10, 'b_bw': 0.2, 'b_speed': 0.2}}
[1848.021606964426]
[3722.994300301292]


## random samples

In [9]:
best_sample_latencies = []
best_sample_num_samples = []
for program_id in p_id:
    for network_id in n_id:
        net_data = generate_network(*list(test_networks[network_id].values()))
        net = FullNetwork(net_data['delay'], net_data['comm_speed'], net_data['speed'], net_data['device_constraints'])
        prg = Program(generate_graph(*list(test_programs[program_id].values())))
        
        constraints = get_placement_constraints(prg, net)
        
        sampled_latencies = []
        for i in range(number_of_tests):
            _, _, random_latency = random_placement(prg, net, constraints, 2 * prg.n_operators)
            sampled_latencies.append(random_latency)
        
        num_samples = []
        min_latencies = []
        for k in np.linspace(1, 2 * prg.n_operators, 15):
            n = int(k)
            num_samples.append(n)
            min_latencies.append([min(samples[:n]) for samples in sampled_latencies])
        
        fig, ax = plt.subplots(1, 1)
        mean = np.array([np.mean(l) for l in min_latencies])
        std = np.array([np.std(l) for l in min_latencies])
        ax.plot(num_samples, mean, color='red', )
        ax.fill_between(num_samples, mean + std, mean - std, alpha=.25, color='red')
        plt.show()

[10, 10, 0, 0, 0.06717514991760254, [2, 8, 0, 8, 8, 9, 9, 0, 0, 8], tensor(660.5646, dtype=torch.float64)]
[10, 10, 1, 0, 0.049162864685058594, [6, 6, 8, 8, 9, 9, 2, 8, 8, 0], tensor(445.9324, dtype=torch.float64)]
[10, 10, 2, 0, 0.05240821838378906, [2, 4, 8, 9, 9, 9, 2, 2, 9, 3], tensor(574.8331, dtype=torch.float64)]
[10, 10, 3, 0, 0.08099818229675293, [6, 6, 6, 2, 2, 1, 8, 0, 2, 7], tensor(647.0161, dtype=torch.float64)]
[10, 10, 4, 0, 0.06606602668762207, [3, 3, 3, 1, 8, 3, 8, 8, 9, 3], tensor(485.9895, dtype=torch.float64)]
[10, 20, 0, 0, 18.91101622581482, [0, 0, 2, 9, 2, 2, 9, 8, 0, 8, 0, 0, 8, 8, 0, 8, 8, 8, 4, 9], tensor(1316.5987, dtype=torch.float64)]
[10, 20, 1, 0, 14.80750298500061, [6, 6, 2, 8, 8, 2, 8, 2, 8, 8, 9, 8, 8, 8, 1, 9, 2, 2, 6, 1], tensor(1199.4469, dtype=torch.float64)]
[10, 20, 2, 0, 26.272990226745605, [3, 9, 2, 8, 2, 8, 8, 2, 2, 2, 8, 8, 9, 9, 2, 8, 2, 9, 8, 7], tensor(1180.6202, dtype=torch.float64)]
[10, 20, 3, 0, 54.67387413978577, [8, 8, 8, 2, 1, 2, 2,

In [10]:
noises = [0, 0.01, 0.05, 0.1]
n_episodes = 20
n_iter = 50

file = open('data/random_op_est_data.pk', 'wb')

for ndev in n_devices:
    network = networks[ndev]
    for nop in n_operators:
        for seed, program in enumerate(programs[ndev][nop]):
            mapping = program.random_mapping()
            for noise in noises:
                r = {'n_devices': ndev, 
                    'n_operators': nop, 
                    'seed': seed, 
                    'noise': noise, 
                    'episodes': []}
                print(r)
                for i in range(n_episodes):
                    s = time.time()
                    random_mapping, random_latency, random_latency_trace = random_op_est_dev(program, network, mapping, n_iter, noise)
                    t = time.time() - s
                    r['episodes'].append([t, random_mapping, random_latency, random_latency_trace])
                    print(f'{t}  {random_mapping}')
                pickle.dump(r, file)

{'n_devices': 10, 'n_operators': 10, 'seed': 0, 'noise': 0, 'episodes': []}
0.8202738761901855  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
0.732633113861084  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
0.6822371482849121  [2, 8, 0, 8, 8, 1, 1, 0, 0, 8]
1.2688038349151611  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
0.9286208152770996  [2, 8, 0, 8, 8, 9, 7, 0, 0, 8]
0.6974351406097412  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
0.6536006927490234  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
0.5854370594024658  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
0.5938379764556885  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
0.6203880310058594  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
0.6049389839172363  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
0.651292085647583  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
0.6475698947906494  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
0.689471960067749  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
0.5093750953674316  [2, 8, 5, 8, 8, 9, 9, 0, 0, 8]
0.5935120582580566  [2, 8, 3, 8, 8, 9, 9, 0, 0, 8]
0.6311862468719482  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
0.695451021194458  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
0.69458699

0.8181767463684082  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
0.8269438743591309  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
0.9017248153686523  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
0.9171528816223145  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
0.926041841506958  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
0.8469698429107666  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
1.0706548690795898  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
0.798112154006958  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
0.7775421142578125  [6, 6, 8, 6, 9, 9, 9, 8, 8, 0]
0.8992340564727783  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
{'n_devices': 10, 'n_operators': 10, 'seed': 2, 'noise': 0, 'episodes': []}
0.6664299964904785  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
0.6604888439178467  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
0.7024848461151123  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
0.7529678344726562  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
0.7014384269714355  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
0.6889529228210449  [2, 2, 8, 9, 9, 7, 2, 2, 9, 3]
0.6451928615570068  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
0.669762134552002  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
0.6777992

0.8690872192382812  [6, 6, 6, 8, 8, 4, 8, 1, 3, 7]
0.7320339679718018  [6, 6, 6, 8, 8, 4, 8, 1, 8, 7]
0.7349429130554199  [6, 6, 6, 8, 8, 4, 8, 1, 3, 7]
0.7232801914215088  [6, 6, 6, 8, 8, 4, 8, 1, 8, 7]
1.1025230884552002  [6, 6, 6, 8, 8, 4, 8, 1, 8, 7]
0.7182071208953857  [6, 6, 6, 8, 8, 4, 8, 1, 8, 7]
0.6728000640869141  [6, 6, 6, 8, 8, 4, 8, 1, 8, 7]
0.7534592151641846  [6, 6, 6, 8, 8, 4, 8, 1, 8, 7]
0.8344881534576416  [6, 6, 6, 8, 8, 4, 8, 1, 8, 7]
0.7451040744781494  [6, 6, 6, 8, 8, 4, 8, 1, 8, 7]
0.6787679195404053  [6, 6, 6, 8, 8, 4, 8, 1, 8, 7]
0.7277390956878662  [6, 6, 6, 8, 8, 4, 8, 1, 8, 7]
0.7780818939208984  [6, 6, 6, 8, 8, 4, 8, 1, 8, 7]
0.8342108726501465  [6, 6, 6, 8, 8, 4, 8, 1, 8, 7]
0.7573621273040771  [6, 6, 6, 8, 8, 4, 8, 1, 8, 7]
0.6956357955932617  [6, 6, 6, 8, 8, 4, 8, 1, 8, 7]
0.6668939590454102  [6, 6, 6, 8, 8, 4, 8, 1, 8, 7]
0.6388049125671387  [6, 6, 6, 8, 8, 4, 8, 1, 8, 7]
0.7624859809875488  [6, 6, 6, 8, 8, 4, 8, 1, 8, 7]
0.9709579944610596  [6, 6, 6, 8

3.938645839691162  [0, 0, 8, 9, 9, 9, 9, 8, 0, 8, 0, 0, 8, 8, 0, 8, 8, 8, 4, 9]
3.847501754760742  [0, 0, 8, 9, 9, 9, 9, 8, 0, 8, 0, 0, 8, 8, 0, 1, 8, 8, 4, 9]
3.804108142852783  [0, 0, 8, 9, 9, 9, 9, 8, 3, 8, 0, 0, 8, 8, 0, 8, 8, 8, 4, 9]
4.029910564422607  [0, 0, 8, 9, 9, 9, 7, 8, 0, 8, 0, 0, 8, 8, 8, 8, 8, 8, 4, 9]
3.6243510246276855  [0, 0, 8, 9, 9, 6, 6, 8, 0, 8, 0, 0, 9, 9, 0, 8, 8, 8, 4, 9]
3.358332872390747  [0, 0, 9, 9, 9, 9, 9, 8, 0, 8, 0, 0, 8, 8, 0, 8, 8, 8, 4, 9]
3.6833181381225586  [0, 0, 8, 9, 9, 9, 9, 8, 0, 8, 0, 0, 8, 9, 8, 8, 8, 8, 4, 9]
{'n_devices': 10, 'n_operators': 20, 'seed': 0, 'noise': 0.05, 'episodes': []}
3.21294903755188  [0, 0, 8, 9, 9, 9, 9, 8, 0, 8, 0, 0, 8, 8, 8, 8, 8, 8, 3, 9]
3.4274709224700928  [0, 0, 8, 9, 9, 9, 7, 8, 0, 7, 0, 0, 8, 8, 0, 8, 8, 8, 4, 9]
3.2027060985565186  [0, 0, 8, 7, 7, 9, 7, 8, 0, 8, 0, 0, 8, 8, 0, 8, 8, 8, 4, 9]
3.9308559894561768  [0, 0, 8, 9, 9, 9, 9, 8, 0, 8, 0, 0, 8, 8, 0, 8, 8, 8, 4, 9]
3.315294027328491  [0, 0, 9, 9, 7, 9,

3.7529332637786865  [6, 6, 9, 8, 8, 3, 8, 3, 3, 8, 9, 8, 0, 0, 0, 9, 9, 9, 6, 1]
3.928287982940674  [6, 6, 9, 8, 8, 8, 8, 9, 9, 8, 9, 8, 8, 8, 0, 9, 9, 2, 6, 1]
3.6599433422088623  [6, 6, 9, 8, 8, 8, 8, 9, 9, 6, 9, 8, 8, 8, 0, 9, 9, 9, 6, 1]
4.001973867416382  [6, 6, 9, 8, 8, 8, 8, 9, 3, 8, 9, 8, 0, 0, 0, 9, 9, 9, 6, 1]
4.074525833129883  [6, 6, 9, 8, 8, 3, 8, 3, 3, 8, 9, 8, 8, 8, 8, 9, 9, 9, 6, 1]
3.9726479053497314  [6, 6, 9, 8, 3, 3, 8, 3, 3, 8, 9, 8, 0, 0, 0, 9, 9, 9, 6, 1]
3.68635892868042  [6, 6, 9, 8, 8, 8, 8, 9, 9, 8, 9, 8, 8, 0, 8, 9, 9, 9, 6, 1]
4.302108287811279  [6, 6, 9, 8, 8, 3, 8, 3, 3, 8, 9, 8, 8, 8, 0, 9, 9, 9, 6, 1]
3.7344610691070557  [6, 6, 9, 8, 8, 8, 8, 9, 9, 8, 9, 8, 0, 0, 0, 9, 9, 9, 6, 1]
{'n_devices': 10, 'n_operators': 20, 'seed': 1, 'noise': 0.1, 'episodes': []}
3.45434308052063  [6, 6, 9, 0, 0, 0, 0, 9, 9, 8, 9, 8, 0, 0, 0, 9, 9, 9, 6, 1]
4.1003100872039795  [6, 6, 9, 0, 0, 0, 0, 3, 4, 8, 9, 8, 0, 0, 0, 9, 9, 9, 6, 1]
3.5868778228759766  [6, 6, 9, 8, 8, 8, 

...

7.661834955215454  [1, 22, 22, 15, 10, 6, 24, 24, 16, 5, 16, 24, 10, 24, 1, 16, 24, 13, 10, 24, 24, 28, 24, 22, 15, 24, 28, 13, 10, 7]
8.297269105911255  [1, 1, 10, 15, 10, 16, 24, 21, 5, 5, 28, 24, 10, 24, 28, 16, 24, 13, 10, 24, 24, 4, 24, 24, 15, 24, 28, 13, 2, 7]
8.303094148635864  [1, 1, 10, 12, 12, 16, 24, 21, 16, 5, 16, 24, 16, 24, 28, 16, 6, 13, 10, 24, 24, 28, 24, 22, 12, 24, 8, 13, 10, 7]
8.200225591659546  [1, 1, 22, 2, 2, 16, 24, 24, 16, 5, 28, 24, 10, 24, 28, 16, 24, 13, 10, 24, 24, 28, 24, 22, 22, 24, 8, 13, 2, 7]
7.706689119338989  [1, 1, 10, 2, 2, 16, 24, 24, 16, 5, 28, 24, 18, 15, 28, 16, 24, 13, 10, 24, 24, 4, 24, 22, 10, 24, 28, 13, 2, 7]
8.249968767166138  [1, 1, 12, 12, 10, 16, 24, 24, 16, 19, 16, 24, 10, 24, 1, 16, 24, 13, 10, 24, 24, 28, 17, 17, 12, 24, 28, 13, 2, 7]
8.199143886566162  [1, 1, 10, 10, 10, 16, 24, 24, 16, 5, 16, 24, 16, 15, 28, 16, 24, 17, 10, 17, 24, 28, 24, 24, 10, 18, 28, 26, 10, 7]
7.41688084602356  [1, 1, 10, 15, 15, 16, 24, 24, 5, 5, 28, 24, 

8.248585939407349  [0, 0, 24, 15, 8, 11, 10, 11, 23, 9, 24, 24, 24, 2, 17, 2, 16, 17, 22, 22, 24, 16, 24, 27, 24, 2, 10, 24, 10, 8]
7.966113805770874  [0, 19, 24, 24, 8, 8, 18, 18, 23, 16, 24, 24, 10, 8, 24, 2, 16, 24, 24, 2, 24, 16, 13, 0, 24, 8, 10, 22, 10, 8]
8.43211317062378  [0, 0, 14, 24, 8, 8, 10, 11, 11, 16, 24, 24, 15, 2, 12, 2, 16, 23, 22, 22, 24, 16, 24, 0, 5, 2, 10, 24, 10, 8]
8.285724878311157  [0, 0, 24, 24, 8, 8, 18, 18, 18, 9, 24, 24, 10, 2, 24, 0, 16, 24, 24, 2, 24, 16, 24, 0, 24, 2, 10, 24, 24, 8]
8.712251901626587  [0, 0, 24, 24, 8, 18, 10, 10, 24, 16, 24, 8, 24, 8, 12, 0, 16, 12, 12, 22, 24, 19, 24, 0, 24, 2, 10, 24, 10, 8]
9.30843210220337  [0, 19, 24, 24, 18, 18, 18, 18, 23, 16, 15, 24, 15, 18, 24, 2, 16, 24, 24, 2, 24, 16, 24, 0, 24, 2, 10, 24, 24, 8]
9.641808032989502  [0, 0, 24, 15, 8, 8, 10, 11, 11, 16, 24, 24, 24, 8, 12, 2, 19, 12, 12, 2, 13, 16, 24, 27, 24, 2, 25, 24, 17, 8]
8.856551885604858  [0, 0, 14, 24, 8, 8, 10, 10, 11, 9, 24, 24, 10, 2, 12, 2, 16, 12,

7.901181936264038  [1, 17, 24, 28, 24, 16, 2, 2, 14, 28, 5, 24, 28, 25, 3, 24, 28, 4, 17, 17, 24, 24, 24, 28, 12, 5, 25, 17, 28, 15]
8.091591119766235  [1, 17, 17, 15, 24, 5, 2, 12, 24, 28, 28, 9, 28, 24, 28, 24, 28, 17, 17, 18, 24, 24, 24, 28, 17, 17, 24, 17, 28, 15]
8.485870838165283  [1, 8, 24, 28, 24, 16, 2, 2, 24, 28, 28, 28, 28, 12, 3, 24, 28, 17, 26, 26, 24, 24, 24, 28, 12, 5, 24, 17, 28, 15]
8.183818101882935  [1, 8, 24, 15, 24, 16, 2, 2, 24, 5, 9, 9, 28, 24, 28, 24, 28, 17, 26, 26, 24, 21, 21, 28, 17, 18, 24, 17, 28, 15]
8.71453595161438  [1, 17, 22, 28, 24, 16, 2, 12, 15, 28, 9, 10, 28, 12, 28, 21, 28, 17, 17, 17, 21, 21, 21, 28, 17, 17, 25, 17, 28, 15]
7.708960056304932  [1, 17, 17, 15, 24, 5, 12, 12, 15, 5, 5, 10, 28, 24, 3, 21, 28, 17, 17, 17, 24, 24, 21, 3, 17, 18, 24, 17, 3, 15]
8.683851957321167  [1, 17, 17, 28, 24, 16, 2, 2, 24, 28, 28, 10, 28, 25, 3, 24, 28, 17, 17, 17, 24, 21, 21, 28, 17, 18, 25, 17, 3, 15]
8.495520114898682  [1, 17, 24, 15, 24, 16, 12, 12, 24, 5, 5,

6.115869998931885  [23, 14, 0, 22, 24, 21, 22, 24, 24, 16, 17, 16, 23, 22, 24, 7, 17, 16, 24, 17, 24, 24, 10, 17, 17, 24, 24, 17, 17, 15]
5.881466865539551  [23, 23, 24, 22, 24, 10, 24, 24, 3, 3, 17, 24, 23, 22, 24, 24, 17, 16, 24, 17, 24, 17, 10, 17, 21, 24, 24, 17, 17, 15]
6.637836217880249  [23, 23, 24, 22, 24, 10, 24, 6, 24, 16, 17, 12, 10, 22, 28, 24, 17, 16, 24, 2, 2, 10, 10, 17, 17, 24, 19, 11, 17, 15]
6.131957054138184  [23, 23, 24, 22, 24, 21, 24, 6, 3, 16, 17, 16, 10, 22, 28, 24, 17, 16, 24, 17, 24, 24, 10, 17, 17, 24, 27, 17, 17, 15]
6.210011959075928  [23, 23, 24, 22, 24, 10, 24, 24, 24, 3, 17, 24, 10, 22, 28, 7, 17, 16, 19, 17, 24, 17, 10, 26, 17, 24, 19, 17, 17, 15]
5.75923490524292  [23, 23, 24, 22, 24, 6, 24, 6, 24, 16, 17, 16, 10, 22, 24, 24, 17, 16, 24, 17, 24, 24, 26, 26, 21, 21, 19, 17, 17, 15]
6.476556062698364  [23, 23, 24, 22, 6, 6, 24, 6, 24, 16, 17, 16, 10, 22, 24, 24, 17, 16, 24, 17, 28, 17, 10, 26, 21, 24, 24, 17, 17, 15]
6.302515029907227  [23, 23, 24, 22, 2

8.098167181015015  [23, 23, 0, 20, 24, 10, 22, 29, 24, 16, 17, 16, 10, 22, 24, 24, 17, 16, 24, 17, 24, 10, 10, 26, 17, 24, 24, 17, 17, 15]
7.540065050125122  [23, 14, 24, 22, 6, 21, 22, 6, 24, 16, 17, 16, 10, 22, 28, 24, 17, 16, 24, 17, 24, 24, 10, 26, 21, 21, 24, 17, 17, 15]
8.068195819854736  [23, 23, 24, 22, 6, 6, 24, 29, 7, 16, 17, 16, 10, 22, 24, 24, 17, 16, 24, 17, 2, 10, 10, 17, 21, 21, 24, 17, 3, 15]
7.821141004562378  [23, 23, 24, 22, 24, 10, 24, 29, 3, 16, 17, 16, 23, 22, 24, 24, 17, 16, 24, 17, 24, 24, 10, 26, 17, 24, 27, 17, 17, 15]
8.017539739608765  [23, 23, 24, 20, 24, 10, 22, 24, 3, 16, 17, 24, 10, 22, 24, 24, 17, 16, 24, 17, 2, 24, 10, 26, 17, 21, 27, 17, 3, 15]
7.505333185195923  [23, 23, 24, 20, 24, 10, 24, 24, 3, 16, 17, 16, 10, 22, 24, 7, 17, 16, 24, 17, 24, 10, 10, 26, 17, 24, 24, 17, 3, 15]
7.988206148147583  [23, 23, 0, 22, 6, 6, 24, 6, 3, 18, 17, 24, 10, 22, 28, 24, 17, 16, 24, 2, 2, 24, 26, 26, 17, 24, 24, 17, 3, 15]
7.570553779602051  [23, 23, 24, 22, 24, 10,

In [4]:
def train(env,
          agent,
          program_id, 
          network_id,
          init_mapping,
          episodes,
          samples_to_ops_ratio=50,
          update_op_net=True,
          update_dev_net=False,
          noise=0):
    op_rewards = []

    map_records = [init_mapping]
    lat_records = []
    act_records = []

    program = env.programs[program_id]
    network = env.networks[network_id]
    
    records = []

    mask_dev = torch.zeros(network.n_devices).to(device)



    for i in range(episodes):
        cur_mapping = init_mapping.copy()
        last_latency, path, G_stats = env.simulate(program_id, network_id, init_mapping, noise)

        latencies = [last_latency]
        actions = []
        rewards = []
        ep_reward = 0

        mask_op = torch.ones(program.n_operators).to(device)
        mask_op[0] = mask_op[-1] = 0
        
        start_time = time.time()

        for t in range(samples_to_ops_ratio):
            temp_mapping = cur_mapping.copy()
            g = env.get_cardinal_graph(program_id, network_id, temp_mapping, path, G_stats).to(device)
            s = agent.op_selection(g, mask_op)

            action = agent.dev_selection_est(program, network, cur_mapping, G_stats, s, program.placement_constraints[s])
            
            cur_mapping[s] = action
            latency, path, G_stats = env.simulate(program_id, network_id, cur_mapping, noise)
            reward = (last_latency - latency)/10
            last_latency = latency
            
            rewards.append(reward)
            latencies.append(latency)
            actions.append([s, action])
            agent.saved_rewards.append(reward)
            ep_reward = reward + ep_reward * agent.gamma

        agent.finish_episode(update_op_net, update_dev_net, use_baseline=True)
        end_time = time.time() - start_time
        records.append([end_time, cur_mapping, ep_reward, latencies, rewards, actions])
        print(f'{end_time}  {ep_reward}  {cur_mapping}')


    return records


In [5]:
program_idx_dict = {}
network_idx_dict = {10: 0, 20: 1, 30: 2}
p_list = []
idx = 0
for n in n_devices:
    program_idx_dict[n] = {}
    for m in n_operators:
        program_idx_dict[n][m] = {}
        for seed in range(5):
            program_idx_dict[n][m][seed] = idx
            idx += 1
            p_list.append(programs[n][m][seed])

env = PlacementEnv([networks[n] for n in n_devices ], p_list)
agent = PlacementAgent(env.get_node_feature_dim(),
                       env.get_edge_feature_dim(),
                       10, 10, gamma = 0.9)

file = open('data/rl_op_data.pk', 'wb')

for ndev in n_devices:
    network_id = network_idx_dict[ndev]
    for nop in n_operators:
        for seed, program in enumerate(programs[ndev][nop]):
            mapping = program.random_mapping()
            program_id = program_idx_dict[ndev][nop][seed]
            for noise in noises:
                r = {'n_devices': ndev, 
                    'n_operators': nop, 
                    'seed': seed, 
                    'noise': noise}
                print(r)
                record = train(env, agent, program_id, network_id, mapping, n_episodes, n_iter, noise=noise)
                r['episodes'] = record
                pickle.dump(r, file)

{'n_devices': 10, 'n_operators': 10, 'seed': 0, 'noise': 0}


/Users/yihu/opt/anaconda3/envs/default/lib/python3.8/site-packages/dgl/base.py:45: DGLWarning: The input graph for the user-defined edge function does not contain valid edges
  return warnings.warn(message, category=category, stacklevel=1)


6.824283123016357  0.36341756556886123  [2, 8, 3, 8, 8, 9, 9, 0, 3, 8]
6.677567005157471  1.215889609559029  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
6.48790717124939  1.1820201592145445  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
6.766561985015869  0.5453196396237502  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
6.419065237045288  0.630289191576816  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
6.428569793701172  0.5781507184169019  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
6.151050806045532  0.5982018442960957  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
5.91358494758606  0.6627043199709947  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
5.94224214553833  0.8279120356066898  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
5.557631015777588  0.593682264442991  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
6.135962963104248  1.5810658626838991  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
5.828226804733276  1.1034602819083867  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
6.148659944534302  0.5682957597076549  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
5.600543975830078  0.6183487073884281  [2, 8, 0, 8, 8, 9, 9, 0, 0, 8]
5.973176956176758  2.0486

5.8257060050964355  0.37746525505194584  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
6.335193872451782  0.3556300152345259  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
6.097543001174927  1.8581243939841932  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
6.585005044937134  0.4706180621004056  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
6.191680669784546  0.8020786196849878  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
5.87201189994812  0.26692902830548815  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
6.6624181270599365  0.6867972940847505  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
{'n_devices': 10, 'n_operators': 10, 'seed': 1, 'noise': 0.05}
6.6534740924835205  0.6759745685953418  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
6.488687038421631  1.623277430867327  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
6.286393165588379  0.6564369606892844  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
6.519564151763916  0.34185430465122724  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
6.519971132278442  1.1653609875565112  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
6.022166013717651  1.0251904464185848  [6, 6, 8, 8, 9, 9, 9, 8, 8, 0]
6.633234262466431  0.95

6.565963983535767  -0.9908783930300247  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
6.02000617980957  2.183820064794527  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
6.346670866012573  0.17384643832600322  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
6.287812948226929  2.6636519512033328  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
6.691667318344116  2.9496171586184197  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
6.680564880371094  4.215943611968995  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
5.834894895553589  -0.3320963867029557  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
6.366426944732666  -0.24489900839948797  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
5.843625068664551  0.40024269921172206  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
6.3690760135650635  -0.21822792466037444  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
5.874493837356567  1.7643687304929154  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
6.24032998085022  1.7333398301872234  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
6.051612138748169  -0.30499647485741077  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
6.4242753982543945  -0.6950310785876073  [2, 2, 8, 9, 9, 9, 2, 2, 9, 3]
7.106473922

...

22.80137801170349  3.4783483388815783  [1, 1, 10, 10, 10, 16, 24, 24, 5, 5, 28, 24, 16, 24, 28, 16, 24, 17, 10, 24, 24, 28, 17, 22, 10, 24, 8, 26, 2, 7]
22.88557481765747  4.175399234038054  [1, 1, 10, 10, 10, 16, 24, 24, 5, 5, 28, 24, 16, 24, 28, 16, 24, 17, 10, 24, 24, 28, 17, 22, 10, 24, 8, 26, 2, 7]
22.04981303215027  6.167065463756085  [1, 1, 10, 10, 10, 16, 24, 24, 5, 5, 28, 24, 16, 24, 28, 16, 24, 17, 10, 24, 24, 28, 17, 22, 10, 24, 8, 26, 2, 7]
22.129405975341797  -3.3654050335932757  [1, 1, 10, 10, 10, 16, 24, 24, 5, 5, 28, 24, 16, 24, 28, 16, 24, 17, 10, 24, 24, 28, 17, 22, 10, 24, 8, 26, 2, 7]
22.125804901123047  5.983853307232004  [1, 1, 10, 10, 10, 16, 24, 24, 5, 5, 28, 24, 16, 24, 28, 16, 24, 17, 10, 24, 24, 28, 17, 22, 10, 24, 8, 26, 2, 7]
23.432454109191895  -2.9752048349278173  [1, 1, 10, 10, 10, 16, 24, 24, 5, 5, 28, 24, 16, 24, 28, 16, 24, 17, 10, 24, 24, 28, 17, 22, 10, 24, 8, 26, 2, 7]
{'n_devices': 30, 'n_operators': 30, 'seed': 2, 'noise': 0}
23.237141132354736  

25.123210191726685  -18.32487547113468  [0, 0, 24, 24, 8, 11, 10, 11, 23, 16, 24, 8, 10, 8, 17, 0, 16, 23, 22, 2, 13, 16, 13, 0, 5, 8, 10, 22, 17, 8]
23.98650598526001  -2.1211809140884395  [0, 0, 14, 24, 8, 11, 10, 11, 23, 16, 24, 8, 10, 8, 17, 0, 16, 23, 22, 2, 13, 16, 13, 0, 5, 8, 10, 22, 17, 8]
24.114032983779907  0.5097343811583717  [0, 0, 14, 24, 8, 11, 10, 11, 23, 16, 24, 8, 10, 8, 17, 0, 16, 23, 22, 2, 13, 16, 13, 0, 5, 8, 10, 22, 17, 8]
23.456319093704224  2.870223656942279  [0, 0, 14, 24, 8, 11, 10, 11, 23, 16, 24, 8, 10, 8, 17, 0, 16, 23, 22, 2, 13, 16, 13, 0, 24, 8, 10, 22, 17, 8]
23.38692617416382  -5.842725095383758  [0, 0, 24, 24, 8, 11, 10, 11, 23, 16, 24, 8, 10, 8, 17, 0, 16, 23, 22, 2, 13, 16, 13, 0, 5, 8, 10, 22, 17, 8]
22.777218103408813  2.4564249374252416  [0, 0, 14, 24, 8, 11, 10, 11, 23, 16, 24, 8, 10, 8, 17, 0, 16, 23, 22, 2, 13, 16, 13, 0, 5, 8, 10, 22, 17, 8]
22.901072025299072  -0.5280603747416109  [0, 0, 14, 24, 8, 11, 10, 11, 23, 16, 24, 8, 10, 8, 17, 0, 1

21.71450185775757  0.3803626252766592  [1, 17, 17, 28, 24, 5, 2, 14, 14, 5, 5, 10, 16, 10, 28, 24, 22, 17, 26, 26, 24, 24, 24, 28, 17, 5, 10, 17, 26, 15]
24.064105987548828  0.5363334623355167  [1, 17, 17, 28, 24, 5, 2, 14, 14, 5, 5, 10, 16, 10, 28, 24, 22, 17, 26, 17, 24, 24, 24, 28, 17, 5, 10, 17, 26, 15]
22.409727096557617  -2.648268202318498  [1, 17, 17, 28, 24, 5, 2, 14, 14, 5, 5, 10, 16, 10, 28, 24, 22, 17, 26, 26, 24, 24, 24, 28, 17, 5, 10, 17, 26, 15]
22.359296083450317  1.7534182051404026  [1, 17, 17, 28, 24, 5, 2, 2, 14, 5, 5, 10, 16, 10, 28, 24, 22, 17, 26, 26, 24, 24, 24, 28, 17, 5, 10, 17, 26, 15]
24.18966794013977  1.0498330641563487  [1, 17, 17, 28, 24, 5, 2, 14, 14, 5, 5, 10, 16, 10, 28, 24, 22, 17, 26, 26, 24, 24, 24, 28, 17, 5, 10, 17, 26, 15]
22.330565929412842  0.22118316571539332  [1, 17, 17, 28, 24, 5, 2, 14, 14, 5, 5, 10, 16, 10, 28, 24, 22, 17, 26, 17, 24, 24, 24, 3, 17, 5, 10, 17, 26, 15]
22.397018671035767  -0.22807383023110306  [1, 17, 17, 28, 24, 5, 2, 14, 1

22.403076171875  -0.5293685592462114  [1, 17, 17, 28, 24, 5, 2, 14, 14, 5, 5, 10, 16, 10, 28, 24, 22, 17, 26, 26, 24, 24, 24, 28, 17, 5, 10, 17, 26, 15]
22.40495276451111  -4.3952987152017595  [1, 17, 17, 28, 24, 5, 2, 14, 14, 5, 5, 10, 16, 10, 28, 24, 22, 17, 26, 26, 24, 24, 24, 28, 17, 5, 10, 17, 26, 15]
21.374731063842773  -1.31598814296476  [1, 17, 17, 28, 24, 16, 2, 2, 14, 5, 5, 10, 16, 10, 28, 24, 22, 17, 26, 26, 24, 24, 24, 28, 17, 5, 10, 17, 26, 15]
21.42512822151184  -2.2440255886604246  [1, 17, 17, 28, 24, 5, 2, 14, 14, 5, 5, 10, 16, 10, 28, 24, 22, 17, 26, 26, 24, 24, 24, 28, 17, 5, 10, 17, 26, 15]
21.951702117919922  1.7697239899517028  [1, 17, 17, 28, 24, 5, 2, 2, 14, 5, 5, 10, 16, 10, 28, 24, 22, 17, 26, 26, 24, 24, 24, 28, 17, 5, 10, 17, 26, 15]
21.997080087661743  3.6357964775149036  [1, 17, 17, 28, 24, 5, 2, 14, 14, 5, 5, 10, 16, 10, 28, 24, 22, 17, 26, 26, 24, 24, 24, 28, 17, 5, 10, 17, 26, 15]
{'n_devices': 30, 'n_operators': 30, 'seed': 4, 'noise': 0}
22.32955694198

22.866605043411255  1.7641083288112693  [23, 23, 24, 22, 24, 10, 22, 24, 24, 16, 17, 24, 23, 22, 28, 24, 17, 16, 24, 2, 28, 10, 26, 26, 17, 24, 27, 11, 17, 15]
22.4129638671875  -1.9230462429415347  [23, 23, 24, 22, 24, 10, 22, 24, 24, 16, 17, 24, 23, 22, 28, 24, 17, 16, 24, 2, 28, 10, 26, 26, 17, 24, 27, 11, 17, 15]
22.489648818969727  -0.6325786184665614  [23, 23, 24, 22, 24, 10, 22, 24, 24, 16, 17, 24, 23, 22, 28, 24, 17, 16, 24, 2, 28, 10, 26, 26, 17, 24, 27, 11, 17, 15]
22.457503080368042  1.9719286156682165  [23, 23, 24, 22, 24, 10, 22, 24, 24, 16, 17, 24, 23, 22, 28, 24, 17, 16, 24, 2, 28, 10, 26, 26, 17, 24, 27, 11, 17, 15]
22.59953022003174  2.7067261819750095  [23, 23, 24, 22, 24, 10, 22, 24, 24, 16, 17, 24, 23, 22, 28, 24, 17, 16, 24, 2, 28, 10, 26, 26, 17, 24, 27, 11, 17, 15]
22.224092960357666  2.6346587377436497  [23, 23, 24, 22, 24, 10, 22, 24, 24, 16, 17, 24, 23, 22, 28, 24, 17, 16, 24, 2, 28, 10, 26, 26, 17, 24, 27, 11, 17, 15]
22.784297943115234  1.7383909853576127  [